In [29]:
import matplotlib.pyplot as plt # data plotting
import numpy as np # linear algebra
from dataclasses import *
import scipy.optimize as optimize
import os
project_path= os.path.dirname(os.path.dirname(os.path.abspath(os.curdir)))
import sys
sys.path.insert(0, project_path+ '/Tools')
sys.path.insert(1, project_path+ '/Systems')
sys.path.insert(2, project_path+ '/Optimal Control Methods/Planning Methods')
from EnforceTyping import enforce_method_typing, EnforceClassTyping
from ParticlesandFields import Field, ClassicalParticle
from DirectCollocation import  DirectCollocation

In [30]:
coulomb_constant = 8.9875e9  # N*m^2/C^2
@dataclass(kw_only=True)
class ElectrostaticField2D(Field):
  """
  A class used to represent a 2D Electrostatic Field

  Attributes
  ----------
  field_sources: dict
      a formatted string to print out what the animal says
  dimensionality: tuple
      a tuple of the dimensionality of the field  

  Methods
  -------
  dynamics(self, observation_position: np.ndarray, time: float) -> np.ndarray:
      Represents the value of the field at any given point(s) or time. 
  potential(self, observation_position: np.ndarray, time: float) -> float:
      Represents the potential due to the field at a given position and/or time  
  potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray, time: float) -> float:
      Represents the potential difference between two positions at a given time in the vector field   
  gradient(self, observation_position: np.ndarray, time: float) -> float:
      Represents the gradient at a given position and/or time in the vector field 
  curl(self, observation_position: np.ndarray, time: float) -> float:
      Represents the curl at a given position and/or time in the vector field 
  divergence(self, observation_position: np.ndarray, time: float) -> float:
      Represents the divergence at a given position and/or time in the vector field
  """
  field_sources: dict
  dimensionality: tuple = (2,)

  def __call__(self, observation_position: np.ndarray) -> np.ndarray:
      return self.dynamics(observation_position)

  def __post_init__(self):
    assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
    for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
        assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle" 

  @enforce_method_typing
  def dynamics(self, observation_position: np.ndarray) -> np.ndarray:
      """
      This function outputs the field strength due to field sources experienced at any given point(s) or time. 
      This determines the physics of the field (a 2D Electricstatic Field in this case)

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric field strength vector at the given position.
      """
      electric_field_vector = np.zeros_like(observation_position)
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_field_vector += (displacement_vectors * field_source.charge) / displacement_magnitude**3
      electric_field_vector = coulomb_constant * electric_field_vector
      return np.round(electric_field_vector, 3)  # N/C or V/m

  @enforce_method_typing
  def potential(self, observation_position: np.ndarray) -> float:
      """
      Calculate the potential (voltage) at a position in the field.

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric potential at the given position.
      """
      electric_potential = 0.0
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_potential += field_source.charge / displacement_magnitude
      electric_potential = coulomb_constant * electric_potential
      return np.round(electric_potential, 3)  # V

  @enforce_method_typing
  def potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray) -> float:
    """
    Calculate the potential difference between the initial position and the final position in the field.

    Args:
        initial_position (np.ndarray): The starting position.
        final_position (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    assert initial_position.shape == self.dimensionality, "initial_position has the wrong dimensions"
    assert final_position.shape == self.dimensionality, "final_position has the wrong dimensions"
    PorentialDifference= self.potential(initial_position)- self.potential(final_position)
    return PorentialDifference

  @enforce_method_typing
  def gradient(self, observation_position: np.ndarray, delta: float= 0.001)->np.ndarray:
    """
    This function returns the derivative of the field at a given point

    Args:
        observation_position (np.ndarray): The position.
        delta (float, optional): The step size. Defaults to 0.001.

    Returns: 
      np.ndarray: The gradient of the field at the given position.
    """
    gradient= np.zeros_like(observation_position)
    for i in range(len(observation_position)):
      di= np.zeros_like(observation_position)
      di[i, ] = di[i, ]+delta
      plusdi= observation_position+ di
      minusdi= observation_position- di
      gradient[i]= (self.dynamics(plusdi)- self.dynamics(minusdi))[i]/ (2* delta)
    return gradient

  @enforce_method_typing
  def plot_field(self, low_bound= -5, high_bound= 5, n_vectors= 50):
    """
    This function plots the 2D electric vector field

    Args:
    low_bound (float, optional): The lower bound of the plot. Defaults to -5.
    high_bound (float, optional): The upper bound of the plot. Defaults to 5.
    n_vectors (int, optional): The number of vectors to plot. Defaults to 50.

    """
    observation_position= np.meshgrid(np.linspace(low_bound, high_bound, n_vectors), 
                                    np.linspace(low_bound, high_bound, n_vectors))
    observation_position= np.stack(observation_position)
    xd, yd = self.dynamics(observation_position)
    xd = xd / np.sqrt(xd**2 + yd**2)
    yd = yd / np.sqrt(xd**2 + yd**2)
    color_aara = np.sqrt(xd**2+ yd**2)
    fig, ax = plt.subplots(1,1)
    cp = ax.quiver(observation_position[0],observation_position[1],xd,yd,color_aara)
    fig.colorbar(cp)
    plt.rcParams['figure.dpi'] = 150
    plt.show()
    

In [31]:
@dataclass
class ChargeInElectricField(EnforceClassTyping, DirectCollocation):
    field: Field
    particle: ClassicalParticle
    ux_max: float
    ux_min: float
    uy_max: float
    uy_min: float
    
    def __post_init__(self):
      self.collocation_points = np.linspace(0, self.final_time, self.n_time_steps)
      self.time_increment= self.collocation_points[1] - self.collocation_points[0]  
      self.optimal_trajectory= self.initial_guess
      self.bounds = [(None, None)]*(4*self.n_time_steps) + [(self.uy_min, self.uy_max)]*self.n_time_steps+ [(self.ux_min, self.ux_max)]*self.n_time_steps

    def objective(self, flattened_trajectory: np.ndarray) -> float:
      position_x, position_y, velocity_x, velocity_y, control_x, control_y = flattened_trajectory.reshape(self.initial_state.size+ np.prod(self.control_dims), self.n_time_steps)
      # positions= np.stack([position_x, position_y], axis=1)
      # field_force= self.particle.charge* np.array([self.field.dynamics(position) for position in positions])
      # field_force_x, field_force_y= field_force.T
      # work= sum((field_force_x**2 +field_force_y**2)/2)
      # effort= 0.0
      # for t in range(self.n_time_steps-1):
      #   effort -= (self.time_increment/2)* ((control_x[t]**2 +control_y[t]**2+ control_x[t+1]**2 +control_y[t+1]**2))
      effort= sum((control_x**2 +control_y**2))/2
      distance= (sum((velocity_x**2 +velocity_y**2))*self.time_increment) 
      return distance
    
    def state_dynamics(self, state: np.ndarray, time: float, control: np.ndarray):
      """
      Compute the dynamics of the particle's state.

      Parameters:
      state (np.ndarray): The current state of the particle [x, y, vx, vy].
      time (float): The current time.
      control_force (np.ndarray): The external control force applied to the particle.

      Returns:
      np.ndarray: The derivative of the state [vx, vy, ax, ay].
      """
      velocity = state[2:]
      acceleration = (self.particle.charge * self.field.dynamics(state[:2]) + control) / self.particle.mass
      return np.concatenate((velocity, acceleration))
    
    def dynamics_constraint(self, flattened_trajectory: np.ndarray):
      position_x, position_y, velocity_x, velocity_y, control_x, control_y = flattened_trajectory.reshape(self.initial_state.size+ np.prod(self.control_dims), self.n_time_steps)
      state_vectors= np.vstack((position_x, position_y, velocity_x, velocity_y)).transpose()
      control_vectors= np.vstack((control_x, control_y)).transpose()
      ceq = []

      for k in range(self.n_time_steps-1):
        time= k* self.time_increment
        state_derivative = self.state_dynamics(state=state_vectors[k], time= time, control=control_vectors[k])
        next_state_derivative = self.state_dynamics(state=state_vectors[k+1], time= time, control=control_vectors[k+1])
        numerical_state_change= (state_vectors[k+1] - state_vectors[k])
        analytical_state_change= (self.time_increment * (state_derivative + next_state_derivative))/2
        ceq.append(numerical_state_change - analytical_state_change)
      return np.array(ceq).flatten()
    
    def boundary_constraints(self, flattened_trajectory: np.ndarray):
      position, angle, velocity, angular_velocity, _, _= flattened_trajectory.reshape(self.initial_state.size+ np.prod(self.control_dims), self.n_time_steps)
      final_state_vector= np.vstack((position, angle, velocity, angular_velocity)).transpose()[-1]
      initial_state_vetor= np.vstack((position, angle, velocity, angular_velocity)).transpose()[0]
      trajectory_boundary= np.concatenate((initial_state_vetor, final_state_vector), axis=None)
      boundary_constraint= np.concatenate((self.initial_state, self.final_state), axis=None)
      return trajectory_boundary-boundary_constraint
    
    def solve(self):
        solution = optimize.minimize(self.objective, 
                                     self.initial_guess, 
                                     method='SLSQP', 
                                     bounds=self.bounds,
                                     constraints=({'type': 'eq', 'fun': self.dynamics_constraint}, 
                                                  {'type': 'eq', 'fun': self.boundary_constraints}), 
                                     tol= 1e-9) 
        self.optimal_trajectory= solution.x
    
    def plot_trajectory(self, flattened_trajectory: np.ndarray= None):
      if flattened_trajectory is None:
        flattened_trajectory= self.optimal_trajectory
      position_x, position_y, velocity_x, velocity_y, control_x, control_y = flattened_trajectory.reshape(self.initial_state.size+ np.prod(self.control_dims), self.n_time_steps)

      plt.plot(position_x, position_y)
      plt.xlim(-4,4)
      plt.ylim(-4,4)
      plt.grid(True)
      plt.title('Optimal Path')
      plt.xlabel('x-Axis(m)')
      plt.ylabel('y-Axis(m)')
      plt.show()

      plt.plot(self.collocation_points, control_x, label='Optimal Control x')
      plt.plot(self.collocation_points, control_y, label='Optimal Control y')
      plt.title('Optimal Control')
      plt.xlabel('time(s)')
      plt.ylabel('Control Force(N)')
      plt.legend()
      plt.show()
      
      plt.plot(self.collocation_points, velocity_x, label='Velocity x')
      plt.plot(self.collocation_points, velocity_y, label='Velocity y')
      plt.title('Optimal Velocity')
      plt.xlabel('time(s)')
      plt.ylabel('Velocity(m/s)')
      plt.legend()
      plt.show()


In [ ]:
negative_charge= ClassicalParticle(mass=1e-14, charge= -1e-9)
positive_charge= ClassicalParticle(mass=1e-14, charge= 1e-9)
sources = {"Particle": [negative_charge],
           "Position": [np.array([-1.0, -1.0])]} 

test_electric_field= ElectrostaticField2D(field_sources=sources)

n_time_steps= 100
final_time= 10.0
final_state= np.array([-1.0, 0.0, 0.0, 0.0])#[x, y, v_x, v_y]
initial_state= np.array([0.0, -1.0, 0.0, 0.0])#[x, y, v_x, v_y]
control_dims= (2,)

initial_guess= np.zeros(n_time_steps*(initial_state.size+ np.prod(control_dims)))
initial_guess[:n_time_steps] = np.linspace(initial_state[0], final_state[0], n_time_steps)  # xPosition Initial Guess
initial_guess[n_time_steps:2*n_time_steps] = np.linspace(initial_state[1], final_state[1], n_time_steps)  # yPosition Initial Guess
initial_guess[2*n_time_steps:3*n_time_steps] = np.linspace(initial_state[2], final_state[2], n_time_steps)  # xVelocity Initial Guess
initial_guess[3*n_time_steps:4*n_time_steps] = np.linspace(initial_state[3], final_state[3], n_time_steps)  # yVelocity Initial Guess

charge_in_electric_field= ChargeInElectricField(n_time_steps, 
                                                final_time, 
                                                initial_state, 
                                                final_state, 
                                                control_dims, 
                                                initial_guess, 
                                                test_electric_field, 
                                                positive_charge, 
                                                1e-9, 
                                                -1e-9, 
                                                1e-9, 
                                                -1e-9)
charge_in_electric_field.plot_trajectory(initial_guess)
charge_in_electric_field.objective(initial_guess)

In [ ]:
charge_in_electric_field.solve()
charge_in_electric_field.plot_trajectory()
charge_in_electric_field.objective(charge_in_electric_field.optimal_trajectory)